In [1]:
from typing import Annotated, List, Dict, Any
from typing_extensions import TypedDict
from langgraph.graph import START, END, StateGraph
from langgraph.graph.message import add_messages
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import HumanMessage, AIMessage
from langchain_openai import ChatOpenAI

In [2]:
class State(TypedDict):
    messages: Annotated[List, add_messages]

In [3]:
llm = ChatOpenAI(model="gpt-4o-mini")

In [4]:
def chatbot(state: State) -> Dict[str, Any]:
    messages = state['messages']
    chat = llm.invoke(messages)
    return {'messages': [chat]}

In [6]:
graph = StateGraph(State)
graph.add_node('chatbot', chatbot)
graph.set_entry_point('chatbot')
graph.add_edge('chatbot', END)

app = graph.compile(checkpointer=MemorySaver())


In [8]:
def test(user_input, thread_id):
    config = {"configurable": {"thread_id": thread_id}}
    state: State = {
        "messages": [HumanMessage(content=user_input)]
    }
    state = app.invoke(state, config=config)
    for m in state['messages']:
        print(m.__class__.__name__, ":", m.content)

In [9]:
test('Hi my name is Manmath', 'thread-001')

HumanMessage : Hi my name is Manmath
AIMessage : Hi Manmath! How can I assist you today?


In [10]:
test('What is my name?', 'thread-001')

HumanMessage : Hi my name is Manmath
AIMessage : Hi Manmath! How can I assist you today?
HumanMessage : What is my name?
AIMessage : Your name is Manmath. How can I help you today, Manmath?
